In [ ]:
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split
import glob 
from ultralytics import YOLO
import torch


In [ ]:
os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"

# Verify it works before starting the big training loop
print(f"Torhc version: {torch.__version__}")
print(f"Is GPU detected? {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU still not detected. Check 'video' group permissions.")
    # Stop here if no GPU, otherwise training will crash or be slow on CPU
    exit()

Torhc version: 2.6.0+rocm6.1
Is GPU detected? False
❌ GPU still not detected. Check 'video' group permissions.


: 

export HSA_OVERRIDE_GFX_VERSION=10.3.0

In [4]:
SOURCE_DIR = os.path.join(os.getcwd(), 'YoloFormatLabels')
DEST_DIR = os.path.join(os.getcwd(), 'fdm_manual_dataset')

def organize_dataset():
    images = []
    # FIXED: Added recursive=True and used ** pattern to look in all subfolders
    extensions = ['*.jpg', '*.png', '*.jpeg']
    
    print(f">>> Scanning {SOURCE_DIR} recursively for images...")
    
    for ext in extensions:
        # The ** pattern combined with recursive=True searches all subdirectories
        search_pattern = os.path.join(SOURCE_DIR, '**', ext)
        images.extend(glob.glob(search_pattern, recursive=True))
    
    annotated_pairs = []
    
    print(f">>> Scanning {SOURCE_DIR} for annotated images...")
    
    for img_path in images:
        base_name = os.path.splitext(img_path)[0]
        txt_path = base_name + ".txt"
        
        # We only keep images that have a matching .txt label file
        if os.path.exists(txt_path):
            annotated_pairs.append((img_path, txt_path))
        else:

            print(f"Skipping un-annotated image: {os.path.basename(img_path)}")

    if not annotated_pairs:
        raise ValueError("No annotated (.txt) files found! Did you save them in YOLO format inside the 'manual_annotation_workspace' folder?")

    
    classes_file = os.path.join(SOURCE_DIR, 'classes.txt')
    if not os.path.exists(classes_file):
        raise FileNotFoundError("classes.txt not found. LabelImg creates this automatically when you save. Did you define classes?")
    
    with open(classes_file, 'r') as f:
        class_names = [line.strip() for line in f.readlines() if line.strip()]

    print(f"Found {len(annotated_pairs)} annotated images.")
    print(f"Classes found: {class_names}")

    
    train_pairs, val_pairs = train_test_split(annotated_pairs, test_size=0.1, random_state=42)

    
    if os.path.exists(DEST_DIR):
        try:
            shutil.rmtree(DEST_DIR)
        except:
            pass 
            
    for split, pairs in [('train', train_pairs), ('val', val_pairs)]:
        img_dest = os.path.join(DEST_DIR, 'images', split)
        lbl_dest = os.path.join(DEST_DIR, 'labels', split)
        os.makedirs(img_dest, exist_ok=True)
        os.makedirs(lbl_dest, exist_ok=True)
        
        for img_src, txt_src in pairs:
            shutil.copy(img_src, os.path.join(img_dest, os.path.basename(img_src)))
            shutil.copy(txt_src, os.path.join(lbl_dest, os.path.basename(txt_src)))

    # 5. Create data.yaml
    yaml_content = {
        'path': DEST_DIR,
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(class_names),
        'names': {i: name for i, name in enumerate(class_names)}
    }
    
    yaml_path = os.path.join(DEST_DIR, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f)

    print(f"\n>>> SUCCESS! Dataset ready at: {DEST_DIR}")
    return yaml_path


organize_dataset()

>>> Scanning /home/aipc/Desly/DefectClassification/YoloFormatLabels recursively for images...
>>> Scanning /home/aipc/Desly/DefectClassification/YoloFormatLabels for annotated images...
Found 250 annotated images.
Classes found: ['Cracking', 'Warping', 'Stringing', 'Layer_shifting', 'Off_Platform']

>>> SUCCESS! Dataset ready at: /home/aipc/Desly/DefectClassification/fdm_manual_dataset


'/home/aipc/Desly/DefectClassification/fdm_manual_dataset/data.yaml'

In [5]:
DATASET_YAML = os.path.join(os.getcwd(), 'fdm_manual_dataset', 'data.yaml')
#MODELS = ['yolov5m6u.pt', 'yolov8m.pt', 'yolov8x.pt', 'yolov9e.pt' ]
MODELS = [r"/home/aipc/Desly/DefectClassification/yolov8m.pt"]

HYPERPARAMS = {
    'epochs': 100,      # Set to 5 or 10 if you just want a quick test run!
    'batch': 16,        # Paper used 64, but 16 is safer for typical GPUs
    'imgsz': 640,
    'optimizer': 'Adam', 
    'lr0': 0.0001,
    'lrf': 0.2,
    'momentum': 0.9,
    'weight_decay': 0.0005,
    'augment': True      
}

def train_and_validate(model_toTrain, project_name):
    model_name = model_toTrain.removesuffix(".pt")
    model = YOLO(model_toTrain) 

    print(f">>> Starting training using: {DATASET_YAML}")

    # 2. Train
    # Ultralytics automatically saves the best model during training
    results = model.train(
        data=DATASET_YAML,
        device=0,
        # amp=False,
        project=project_name,
        name=model_name,
        exist_ok=True,        # Overwrite if exists
        verbose=True,
        **HYPERPARAMS
    )

    # 3. Validate
    print("\n>>> Running Validation...")
    metrics = model.val()

    # 4. Print Accuracy Scores
    # In Object Detection, "Accuracy" is measured by mAP (Mean Average Precision)
    print("\n" + "="*30)
    print("      FINAL RESULTS")
    print("="*30)
    
    # mAP@0.5: How accurate the boxes are (Standard Metric)
    map50 = metrics.results_dict['metrics/mAP50(B)']
    # mAP@0.5-0.95: Strict accuracy (Requires very tight boxes)
    map50_95 = metrics.results_dict['metrics/mAP50-95(B)']
    precision = metrics.results_dict['metrics/precision(B)']
    recall = metrics.results_dict['metrics/recall(B)']

    print(f"Precision: {precision:.4f} (Few false alarms?)")
    print(f"Recall:    {recall:.4f} (Found all defects?)")
    print(f"mAP@0.5:   {map50:.4f} (Overall Accuracy Score)")
    print(f"mAP@0.95:  {map50_95:.4f} (Strict Accuracy)")
    
    # 5. Locate Saved Model
    # It is saved automatically, but we print the exact path for you
    best_weight_path = os.path.join(os.getcwd(), project_name, model_name , 'weights', 'best.pt')
    
    print("\n" + "="*30)
    print(f"MODEL SAVED AT: {best_weight_path}")
    print("="*30)

    return best_weight_path

for model in MODELS:
    print(f"Training model : {model}")
    project_name = f"{'fdm_manual_training'}"
    saved_path = train_and_validate(model, project_name)

Training model : /home/aipc/Desly/DefectClassification/yolov8m.pt
>>> Starting training using: /home/aipc/Desly/DefectClassification/fdm_manual_dataset/data.yaml
Ultralytics 8.4.6 🚀 Python-3.12.3 torch-2.6.0+rocm6.1 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.
